# Visale - Action Logement

À destination des jeunes de moins de 30 ans ou des salariés nouvellement embauchés ayant besoin d'un cautionnement pour un logement.

* service-public.fr : https://www.service-public.fr/particuliers/vosdroits/F33453


In [27]:
# Active l'affichage de résultats multiples par cellule
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Situation à analyser

### Cas de test d'aides-simulateur-front

In [28]:
situation_18yo_moving_away = {
    "id": "18yo-moving-away",
    "description": "Un jeune de 18 ans déménage pour des études à l'université",
    "answers": {
        "statut-professionnel": "etudiant",  # activite
        "situation-professionnelle": "sans-emploi",  # dispatchSituationProfessionnelle
        "etudiant-mobilite": "parcoursup-nouvelle-region",  # dispatchEtudiantMobilite
        "boursier": True,  # boursier
        "date-naissance": "2007-03-01",  # date_naissance
        "handicap": False,  # handicap
        "statut-marital": "celibataire",  # statut_marital
        "code-postal-nouvelle-ville": "75101",  # depcom
        "situation-logement": "locataire",  # dispatchSituationLogement
        "type-logement": "logement-meuble",  # dispatchTypeLogement
        "logement-conventionne": True,  # logement_conventionne
        "colocation": False,  # coloc
        "logement-parente-proprietaire": False,  # proprietaire_proche_famille
        "nombre-personnes-logement": 1,  # 🔥 exclude: True
        "loyer-montant-mensuel": 700,  # loyer
        "loyer-montant-charges": 100,  # charges_locatives
        "loyer-difficile-payer": True,  # exclude: True
        "type-revenus": [
            "aucun-autres-revenus"
        ],
        "confirmation-end": [
            "confirmation-end-oui"
        ]
    },
    "questionsToApi": [
        "locapass-eligibilite",
        "mobilite-master-1",
        "mobilite-parcoursup",
        "aide-personnalisee-logement",
        "garantie-visale-eligibilite",
        "garantie-visale"
    ],
    "results": {
        "locapass": 1200,
        "locapass-eligibilite": True,
        "mobilite-master-1": 0,
        "mobilite-master-1-eligibilite": False,
        "mobilite-parcoursup": 500,
        "mobilite-parcoursup-eligibilite": True,
        "aide-personnalisee-logement": 327,
        "aide-personnalisee-logement-eligibilite": True,
        "garantie-visale": 800,
        "garantie-visale-eligibilite": True
    }
}

to_test = situation_18yo_moving_away

### Transcription du cas de test au format de l'API web

In [29]:
from utils_calculate import period, period_last_month  # mois actuel, mois précédent
from notebooks.utils_mapping_simulateur import format_to_openfisca_json


situation_garantie_visale = format_to_openfisca_json(to_test, period)

# from json import dumps
# print(dumps(situation_garantie_visale, indent=2))

## Initialisation de la simulation

In [30]:
from utils_calculate import france_tax_benefit_system, new_simulation

simulation = new_simulation(france_tax_benefit_system, situation_garantie_visale)


## Comparaison des résultats de la simulation et des résultats attendus

In [31]:
from notebooks.utils_calculate import affiche_resultat

affiche_resultat(
    'éligibilité obtenue', 
    "L'éligibilité attendue est égale au résultat obtenu par simulation.", 
    to_test["results"]["garantie-visale-eligibilite"] == simulation.calculate("visale_eligibilite", period)[0],
    simulation.calculate("visale_eligibilite", period)[0]
    )


affiche_resultat(
    'montant max obtenu', 
    "Le montant max attendu est égal au montant obtenu par simulation.", 
    to_test["results"]["garantie-visale"] == simulation.calculate("visale_montant_max", period)[0],
    simulation.calculate("visale_montant_max", period)[0]
    )


❌ L'éligibilité attendue est égale au résultat obtenu par simulation.
éligibilité obtenue : False
✅ Le montant max attendu est égal au montant obtenu par simulation.
montant max obtenu : 800.0


## Analyse des résultats

In [32]:
from notebooks.utils_calculate import (
    est_dans_intervalle, 
    est_strictement_superieur_seuil,
    est_inferieur_ou_egal_plafond,
    est_enum_dans_liste,
    est_pas_enum_dans_liste, 
    date_de_moins_de_nb_mois,
    non_renseignee_ou_egale_valeur_par_defaut
    )

In [33]:
visale_variables = ['visale_eligibilite', 'visale_base_ressources', 'visale_montant_max']
# visale_variables_entities = ['menages', 'menages', 'menages']
# visale_variables_groupes = ['menage_1', 'menage_1', 'menage_1']

for variable_name in visale_variables:
    print(f"{variable_name}: {simulation.calculate(variable_name, period)}")

visale_eligibilite: [False]
visale_base_ressources: [121.7055]
visale_montant_max: [800.]


## Éligibilité

Les conditions pour l'obtention de la Garantie Visale sont multiples : 
* conditions à remplir par le locataire
* conditions à remplir par le logement
* enfin, le propriétaire (bailleur) doit accepter l'acte de cautionnement (hors périmètre pour la présente fiche s'intéressant aux aspects calculatoires).

### Éligibilité locataire

* J’ai entre 18 et 30 ans
* Je suis salarié(e) de plus de 30 ans : 
    * Embauché(e) depuis moins de 6 mois (hors CDI confirmé)
    * OU Gagnant jusqu’à 1500€ nets/mois
    * OU En mobilité professionnelle
    * OU En possession d’une promesse d’embauche de moins de 3 mois
* Je suis éligible au bail mobilité.
* Je suis logé(e) par un organisme d’intermédiation locative

> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/
> Si jeune de moins de 30 ans jusqu'au 31ème anniversaire d'après [ce PDF https://www.morbihan.gouv.fr](https://www.morbihan.gouv.fr/contenu/telechargement/40631/297616/file/VISALE_DEPLIANT_LOCATAIRE_120916.pdf)

####  Cas entre 18 et 30 ans
> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

##### J’ai entre 18 et 30 ans

In [34]:
cas_1_condition_age = est_dans_intervalle(simulation, 'age', 18, 30, "Le locataire doit avoir entre 18 et 30 ans.", 0)


✅ Le locataire doit avoir entre 18 et 30 ans.
age : 18


#### Cas salarié de plus de 30 ans

* Je suis salarié(e) de plus de 30 ans : 
    * Embauché(e) depuis moins de 6 mois (hors CDI confirmé)
    * OU Gagnant jusqu’à 1500€ nets/mois
    * OU En mobilité professionnelle
    * OU En possession d’une promesse d’embauche de moins de 3 mois


> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

In [35]:
# plus de 30 ans

cas_2_condition_age = est_strictement_superieur_seuil(simulation, 'age', 30, "Le locataire a plus de 30 ans.", 0)

❌ Le locataire a plus de 30 ans.
age : 18


In [36]:
# embauché hors CDI confirmé (TODO : clarifier sens de "confirmé")
from openfisca_france.model.revenus.activite.salarie import TypesContrat


cas_2_condition_avoir_contrat_travail = est_pas_enum_dans_liste(
    simulation,
    'contrat_de_travail_type',
    [
        TypesContrat.cdi.name,  # TypesContrat.cdi est valeur par défaut de contrat_de_travail_type 
        TypesContrat.aucun.name, TypesContrat.formation.name  # != locataire embauché
    ],
    TypesContrat.names,
    "Le locataire a un contrat de travail hors CDI confirmé."
    )


# TODO vérifier cohérence avec activite == actif ?

❌ Le locataire a un contrat de travail hors CDI confirmé.
contrat_de_travail_type : cdi


In [37]:
# embauché depuis moins de 6 mois

cas_2_condition_duree_contrat_travail = date_de_moins_de_nb_mois(
    simulation,
    'contrat_de_travail_debut',  # 01/01/1870 par défaut :-D
    6,
    "Le locataire a un contrat de travail depuis moins de 6 mois.")

cas_2_condition_contrat_de_travail_en_cours = non_renseignee_ou_egale_valeur_par_defaut(
    france_tax_benefit_system,
    simulation,
    'contrat_de_travail_fin',
    "Le locataire n'a pas de date de fin de contrat de travail (connue)."
    )


# contrat de travail actuel ? (contrôle date début et date de fin)
print("======= Bilan")

cas_2_condition_embauche_moins_6mois = cas_2_condition_duree_contrat_travail * cas_2_condition_contrat_de_travail_en_cours
duree_depuis_debut_contrat_travail = simulation.calculate('contrat_de_travail_debut', period).astype('timedelta64[M]')
affiche_resultat(
    "contrat de travail actuel",
    "Le locataire est embauché depuis moins de 6 mois.",
    cas_2_condition_embauche_moins_6mois[0],
    f"depuis {duree_depuis_debut_contrat_travail[0]} mois" if duree_depuis_debut_contrat_travail[0] > 0 else "inconnu"  # "inconnue" pour palier à durée négative si date début non renseignée (01/01/1870 par défaut)
    )

❌ Le locataire a un contrat de travail depuis moins de 6 mois.
contrat_de_travail_debut : 1870-01-01
✅ Le locataire n'a pas de date de fin de contrat de travail (connue).
contrat_de_travail_fin : 2099-12-31
======= Bilan
❌ Le locataire est embauché depuis moins de 6 mois.
contrat de travail actuel : inconnu


In [38]:
# salarié gagnant jusqu’à 1500€ nets/mois

# à la demande de visale, la dernière fiche de paie est demandée => période = mois précédant la demande 
cas_2_condition_salaire_net_mois_precedent = est_inferieur_ou_egal_plafond(
    simulation,
    'salaire_net',
    period_last_month,
    1500,
    "Le locataire a un salaire allant jusqu’à 1500€ nets/mois."
)

visale_base_ressources = simulation.calculate('visale_base_ressources', period)  # inclut le salaire_net du mois précédent
print(f"[{period}] visale_base_ressources : {visale_base_ressources[0]}")
# TODO identifier la source des revenus

✅ [2025-03] Le locataire a un salaire allant jusqu’à 1500€ nets/mois.
salaire_net : 0.0
[2025-04] visale_base_ressources : 121.70549774169922


In [39]:
# en mobilité professionnelle
# = "faire la demande de la garantie Visale dans les 6 mois qui suivent la mutation (changement de lieu de travail dans la même entreprise ou le même groupe)"
# Src : https://www.service-public.fr/particuliers/vosdroits/F33453

# Justificatif de la mutation (avenant au contrat de travail, attestation de mutation
# de l’employeur, certificat de mutation de l’employeur, lettre de l’employeur,
# attestation de déménagement de l’entreprise). Le contrat de travail initial n’est
# pas demandé.
# Src : https://www.visale.fr/wp-content/uploads/2020/04/Visale-Ressources-Locataire_2020.pdf#page=4

# mutation au mois précédant la demande d'après :
# "Salariés en mutation | Dernière fiche de paie"
# Src : https://www.visale.fr/wp-content/uploads/2020/04/Visale-Ressources-Locataire_2020.pdf#page=5

mobilite_professionnelle = True  # TODO ajouter à openfisca-france et au questionnaire aides-simplifiées
cas_2_condition_mobilite_professionnelle = mobilite_professionnelle
affiche_resultat(
    "[⚠️ non calculé] en mobilité professionnelle",
    "Le locataire est en mobilité professionnelle (justifiable par fiche de paie du mois précédent).",
    cas_2_condition_mobilite_professionnelle,
    mobilite_professionnelle
    )

✅ Le locataire est en mobilité professionnelle (justifiable par fiche de paie du mois précédent).
[⚠️ non calculé] en mobilité professionnelle : True


In [40]:
# en possession d’une promesse d’embauche de moins de 3 mois

# "Futur salarié | Promesse d’embauche(*) ou contrat de travail"
# (*) La forme de la promesse d’embauche est libre dès que l’employeur peut être identifié et son intégrité établie. Pour ce faire,
# celle-ci doit contenir les informations relatives à la date de la promesse d’embauche, au salaire, aux conditions du contrat de
# travail (CDD, CDI etc.), à la durée de l’emploi, à la date d’entrée dans l’emploi et à l’employeur (Siret etc.). Une promesse
# d’embauche sous format dématérialisé est acceptée.
# Src : https://www.visale.fr/wp-content/uploads/2020/04/Visale-Ressources-Locataire_2020.pdf#page=4

contrat_de_travail_debut = simulation.calculate('contrat_de_travail_debut', period)
duree_depuis_debut_contrat_travail = contrat_de_travail_debut.astype('timedelta64[M]')
contrat_de_travail_debut_default_value = france_tax_benefit_system.get_variable('contrat_de_travail_debut').default_value

promesse_embauche_moins_3mois = True  # TODO ajouter à openfisca-france et au questionnaire aides-simplifiées
cas_2_condition_promesse_embauche = (
    contrat_de_travail_debut != contrat_de_travail_debut_default_value  # avoir un début de contrat de travail défini
  ) * promesse_embauche_moins_3mois

affiche_resultat(
    "[⚠️ non calculé] promesse d'embauche",
    "Le locataire dispose d'une promesse d'embauche de moins de 3 mois.",
    cas_2_condition_promesse_embauche[0],
    cas_2_condition_promesse_embauche[0]
    )

❌ Le locataire dispose d'une promesse d'embauche de moins de 3 mois.
[⚠️ non calculé] promesse d'embauche : False


#### Cas 3 quelque soit l'âge, je suis éligible au bail mobilité.

> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

Le bail mobilité est un bail signé entre le propriétaire (bailleur) d'un logement meublé 
et un locataire considéré comme occupant temporaire (étudiant, salarié en mission temporaire ou en formation professionnelle...). 
D'une durée allant de 1 à 10 mois, il ne peut pas être renouvelé.

Pour quel locataire ?

Seules certaines personnes peuvent être le locataire (ou colocataire) d'un bail mobilité.  
Lorsque le bail débute (date qui peut être différente de celle de la signature du bail), le locataire doit se trouver dans l'une des situations suivantes :

* Formation professionnelle
* Études supérieures
* Contrat d'apprentissage
* Stage
* Engagement volontaire dans le cadre d'un service civique
* Mutation professionnelle
* Mission temporaire dans le cadre de l'activité professionnelle

> Source : https://www.service-public.fr/particuliers/vosdroits/F34759

In [41]:
# TypesContrat.formation ou ContexteActivitePoleEmploi.formation (et TypesActivite.actif par cohérence ?)
# TypesActivite.etudiant
# apprentissage_contrat_debut == True ou apprenti == True
# stagiaire == True (et ajouter contrat_de_travail_type à cdd ou cdi ? ou ajouter catégorie de contrat à TypesContratTravailDureeDeterminee et compter CDI standard ?)
# service_civique == True
# = mobilité professionnelle cas +30 and ci-dessous ?
# ajouter mission à TypesContrat ? mais cumulable avec autre contrat donc distinguer dans une nouvelle catégorie ?

eligible_bail_mobilite = False  # TODO ajouter à openfisca-france et au questionnaire aides-simplifiées
cas_2_condition_eligible_bail_mobilite = eligible_bail_mobilite
affiche_resultat(
    "[⚠️ non calculé] éligible au bail mobilité",
    "Le locataire est éligible au bail mobilité.",
    cas_2_condition_eligible_bail_mobilite,
    eligible_bail_mobilite
    )

❌ Le locataire est éligible au bail mobilité.
[⚠️ non calculé] éligible au bail mobilité : False


### Éligibilité logement

* Peut être meublé ou non
* Doit être ma résidence principale
* Doit avoir un loyer (charges comprises) de 1500€ maximum en Île de France ou de 1300€ maximum sur le reste du territoire
* Doit être décent et respecter le Règlement Sanitaire Départemental (RSD)
* Doit faire l’objet d’un bail (contrat de location)


> Source : https://www.visale.fr/visale-pour-les-locataires/eligibilite/

In [47]:
# peut être meublé ou non
from openfisca_france.model.base import TypesStatutOccupationLogement


# logement possiblement futur mais simulation sur période actuelle 
condition_type_logement = est_enum_dans_liste(
    simulation,
    'statut_occupation_logement',
    [
        TypesStatutOccupationLogement.locataire_meuble.name,
        TypesStatutOccupationLogement.locataire_vide.name
        # TODO + locataire_foyer + locataire_hlm ?
    ],
    TypesStatutOccupationLogement.names,
    "Le logement est meublé ou non."
    )


✅ Le logement est meublé ou non.
statut_occupation_logement : locataire_vide


In [43]:
# doit être ma résidence principale

In [44]:
# doit avoir un loyer (charges comprises) de 1500€ maximum en Île de France ou de 1300€ maximum sur le reste du territoire

In [45]:
# doit être décent et respecter le Règlement Sanitaire Départemental (RSD)

In [46]:
# doit faire l’objet d’un bail (contrat de location)

## Limites

> Source : https://www.visale.fr/wp-content/uploads/2024/08/depliant_visale_locataire_juin24.pdf

Dans la limite de 36 mensualités d’impayés de loyers
et charges pour un logement du parc privé. 
Dans le parc locatif social ou assimilé, prise en charge dans la limite de 9
mois d’impayés de loyers et charges locatives, déduction faite
des aides au logement. 
Dans le parc locatif privé, en cas de
dégradations imputables au locataire, les frais de remise en
état du logement sont couverts, jusqu'à deux mois de loyer et
charges inscrits au bail, en complément du dépôt de garantie
(sauf pour le bail mobilité).

Pour les travailleurs saisonniers, la durée de prise en charge
des impayés est limitée de 1 à 9 mois quel que soit le parc
locatif. Pour les dégradations locatives : prise en charge jusqu’à
deux mois de loyer et charges inscrits au bail, complétée le
cas échéant d’une prise en charge des dommages mobiliers, à
hauteur d’un mois de loyer.